In [44]:
import gekko

In [45]:
m = gekko.GEKKO()

In [46]:
m.options.SOLVER=1  # APOPT is an MINLP solver
# optional solver settings with APOPT
m.solver_options = ['minlp_maximum_iterations 10000', \
                    # minlp iterations with integer solution
                    'minlp_max_iter_with_int_sol 500', \
                    # treat minlp as nlp
                    'minlp_as_nlp 0', \
                    # nlp sub-problem max iterations
                    'nlp_maximum_iterations 50', \
                    # 1 = depth first, 2 = breadth first
                    'minlp_branch_method 1', \
                    # maximum deviation from whole number
                    'minlp_integer_tol 0.05', \
                    # covergence tolerance
                    'minlp_gap_tol 1']

## 各属性英文变量词典

- crit_rate: 暴击率
- crit_dmg: 暴击伤害
- eng_rcg: energy_recharge, 元素充能效率
- big_hp: 大生命（百分比生命加成）

# 圣遗物副词条相关

## 以下是各个受约束变量

In [47]:
crit_rate_cnt = m.Var(7, lb=4, ub=24, integer=True)
crit_dmg_cnt = m.Var(5, lb=5, ub=30, integer=True)
eng_rcg_cnt = m.Var(10, lb=5, ub=30, integer=True)
big_hp_cnt = m.Var(8, lb=5, ub=24, integer=True)

variables = [crit_rate_cnt, crit_dmg_cnt, eng_rcg_cnt, big_hp_cnt]

## 以下是每个有效副词条的期望值

In [48]:
crit_rate_each = 3.305
crit_dmg_each = 6.61
eng_rcg_each = 5.505
big_hp_each = 4.955

## 副词条总值

In [49]:
crit_rate = crit_rate_cnt * crit_rate_each
crit_dmg = crit_dmg_cnt * crit_dmg_each
eng_rcg = eng_rcg_cnt * eng_rcg_each
big_hp = big_hp_cnt * big_hp_each

# 其他加成属性

## 圣遗物主词条加成

In [50]:
atf_small_hp = 4780

atf_big_hp = 46.6
atf_eng_rcg = 0
# atf_big_hp = 0
# atf_eng_rcg = 51.8

atf_water_dmg = 46.6

atf_crit_rate = 31.1

## 圣遗物套装加成

In [51]:
atf_set_eng_rcg = 20
art_set_q_dmg = 41.515

## 武器加成

In [52]:
weap_crit_dmg = 88.2
weap_big_hp = 16
weap_total_dmg = 20

## 人物自身属性

In [53]:
char_hp = 14450
char_crit_rate = 24.2

## 人物技能倍率

In [54]:
# E技能：生命上限的38.4%
e_mul = 0.384
# Q技能：生命上限的385.02%
q_mul = 3.8502

# 计算伤害

## 计算总属性

In [55]:
hp_mul = (big_hp + weap_big_hp + atf_big_hp)/100 # 生命乘区
total_hp = char_hp * (1 + hp_mul) * (1 + 6/100 * 1) + atf_small_hp # 总生命值

total_crit_rate = crit_rate + atf_crit_rate + char_crit_rate # 总暴击率，如果>100则取100

total_crit_dmg = crit_dmg + weap_crit_dmg + 50 # 总爆伤

total_eng_rcg = 100 + (eng_rcg + atf_eng_rcg) * (1 + atf_set_eng_rcg/100) # 总充能

In [56]:
def e_dmg(): 
    result = total_hp * e_mul 
    result *= (1 + (atf_water_dmg + weap_total_dmg)/100 ) # 水伤与武器伤害
    result *= (1 + total_crit_rate/100 * total_crit_dmg/100 ) # 暴击与爆伤
    result *= 0.5 # 防御
    result *= 0.9 # 抗性
    return result


In [57]:
def q_dmg():
    result = total_hp * q_mul
    result *= (1 + (atf_water_dmg + weap_total_dmg + art_set_q_dmg)/100 ) # 水伤、武器、圣遗物伤害
    result *= (1 + total_crit_rate/100 * total_crit_dmg/100 ) # 暴击与爆伤
    result *= 0.5 # 防御
    result *= 0.9 # 抗性
    return result

# 解

In [58]:
# 限制条件
m.Equation(total_eng_rcg >= 160)
m.Equation(sum(variables) == 30)

In [59]:
# 目标函数
m.Maximize(q_dmg())

In [60]:
m.solve()

apm 180.139.214.44_gk_model1 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            5
   Intermediates:            0
   Connections  :            0
   Equations    :            3
   Residuals    :            3
 
 Number of state variables:              5
 Number of total equations: -            2
 Number of slack variables: -            1
 ---------------------------------------
 Degrees of freedom       :              2
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -3.10E+05 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi:

In [ ]:
print('crit_dmg_cnt:', crit_dmg_cnt.value[0])
print('crit_rate_cnt:', crit_rate_cnt.value[0])
print('eng_rcg_cnt:', eng_rcg_cnt.value[0])
print('big_hp_cnt:', big_hp_cnt.value[0])
print('Total Q dmg:', str(-m.options.objfcnval))

crit_dmg_cnt: 5.0
crit_rate_cnt: 10.0
eng_rcg_cnt: 10.0
big_hp_cnt: 5.0
Total Q dmg: 303374.4569


In [ ]:
m.cleanup()

Hello！